In [3]:
import csv
import folium
import random
from folium.plugins import FastMarkerCluster
from geopy.distance import geodesic
from itertools import combinations

archivo_csv = 'dataset.csv'
acceptable_risk= 4
class Node:
    def __init__(self, id, latitude, longitude):
        self.id = id
        self.latitude = latitude
        self.longitude = longitude

class Edge:
    def __init__(self, node1, node2, distance, risk_level):
        self.node1 = node1
        self.node2 = node2
        self.distance = distance
        self.risk_level = risk_level


In [4]:
#Funciones de UFDS, para unir conjuntos
def find_set(sets, node):
    for set in sets:
        if node in set:
            return set
    return None


#Funciones de UFDS, para unir y encontrar conjuntos

def union_sets(sets, set1, set2):
    sets.remove(set1) #remueve un elemento en especifico, set1
    sets.remove(set2)
    sets.append(set1.union(set2)) #une el conjunto 1 y el 2 sin repetir elementos, y crea un nuevo conju


In [5]:
nodes = []
with open(archivo_csv, 'r') as archivo:
    lector_csv = csv.reader(archivo, delimiter=';')
    next(lector_csv)
    for linea in lector_csv:
        id = linea[0]
        latitude = float(linea[3])
        longitude = float(linea[4])
        node = Node(id, latitude, longitude)
        nodes.append(node)



def obtener_numero():
    num_aleatorio = random.randint(1, 100)
    
    if num_aleatorio <= 35:
        return 4  
    else:
        return random.randint(1, 3)  


In [6]:
edges = []
for i in range(len(nodes)):
    for j in range(i + 1, len(nodes)):
        node1 = nodes[i]
        node2 = nodes[j]
        distance = geodesic((node1.latitude, node1.longitude), (node2.latitude, node2.longitude)).kilometers
        risk= obtener_numero()
        edge = Edge(node1, node2, distance,risk)
        edges.append(edge)


In [7]:
edges.sort(key=lambda x: x.distance)

# sets = [
#     {Node1},
#     {Node2},
#     {Node3},
#     {Node4},
#     {Node5}
# ]
sets = []
for node in nodes:
    sets.append({node})

In [8]:
connections = []
for edge in edges:
    if edge.risk_level < acceptable_risk:
        set1 = find_set(sets, edge.node1)
        set2 = find_set(sets, edge.node2)
        if set1 != set2: #verifica que no esten en el mismo conjunto, para no crear bucles
            union_sets(sets, set1, set2)
            connections.append(edge)

In [13]:
m = folium.Map(width='100%', height='100%')

#Se unen los marcadores en grupos usando cluster para que el programa
#vaya con fluidez al navegar por el mapa
fast_marker_cluster = FastMarkerCluster([], name="Cluster").add_to(m)


#Se crean los marcadores, que son la representacion de los nodos en el mapa
for node in nodes:
    fast_marker_cluster.add_child(
        folium.Marker(
            location=[node.latitude, node.longitude],
            popup=f"Name: {node.id}"
        )
    )


#Se crean las conexiones obtenidas al usar kruskal
for connection in connections:
    location1 = (connection.node1.latitude, connection.node1.longitude)
    location2 = (connection.node2.latitude, connection.node2.longitude)
    risk = connection.risk_level
    color = 'green' if risk == 1 else 'blue' if risk == 2 else 'red'
    risk_text = 'Bajo' if risk == 1 else 'Medio' if risk == 2 else 'Alto'
    
    folium.PolyLine(locations=[location1, location2],
                    tooltip=f"Distance: {connection.distance:.2f} km, Risk: {risk_text}",
                    color=color).add_to(m)

m.save("TF.html")
